In [1]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [2]:
# portfolio of stocks
stocks = ['LULU', 'PYPL', 'ADBE', 'FTNT', 'CL', 'ZTS', 'LDOS', 'MNST', 'ADSK', 'MDLZ', 'ISRG', 'MRK', 'NVO', 'UNH', '4528.T', '4568.T', 'BRO', 'DEMANT.CO', 'HRMY', 'HSTM', 'TWLO', 'CRM', 'DUOL', 'GDDY', 'ACN', 'TPL', 'DOCU']

#### Download historical data from yf API

In [3]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker')

# download data to csv
data.to_csv("stocks.csv")

[*********************100%***********************]  27 of 27 completed


#### Download analysis from csv downloaded data

In [4]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("stocks.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# Show the result
df.head()

Ticker     DUOL                               CRM                          \
Price      Open High Low Close Volume        Open        High         Low   
Date                                                                        
2020-01-02  NaN  NaN NaN   NaN    NaN  162.213659  165.330878  161.877202   
2020-01-03  NaN  NaN NaN   NaN    NaN  163.341816  165.034022  163.282443   
2020-01-06  NaN  NaN NaN   NaN    NaN  163.282390  171.862135  162.738112   
2020-01-07  NaN  NaN NaN   NaN    NaN  171.426738  175.028847  170.298606   
2020-01-08  NaN  NaN NaN   NaN    NaN  173.405924  176.988251  172.930925   

Ticker                             ...       4568.T                            \
Price            Close     Volume  ...         Open         High          Low   
Date                               ...                                          
2020-01-02  165.251709  5189300.0  ...          NaN          NaN          NaN   
2020-01-03  164.440262  3205400.0  ...          NaN          NaN          NaN   
2020-01-06  171.644424  8836200.0  ...  2231.428670  2247.340703  2181.196489   
2020-01-07  174.167908  8272700.0  ...  2205.844905  2227.372952  2191.804874   
2020-01-08  175.484070  7225100.0  ...  2206.468866  2220.821051  2164.348772   

Ticker                                     TPL                          \
Price             Close     Volume        Open        High         Low   
Date                                                                     
2020-01-02          NaN        NaN  236.962898  237.255706  233.880882   
2020-01-03          NaN        NaN  241.490816  242.094561  236.069349   
2020-01-06  2195.860596  5632800.0  241.249373  244.509505  240.047962   
2020-01-07  2194.924805  5121900.0  241.747431  242.849213  240.570147   
2020-01-08  2191.180908  5244300.0  243.000237  249.028436  241.490900   

Ticker                            
Price            Close    Volume  
Date                              
2020-01-02  233.980484   24600.0  
2020-01-03  241.249329   62400.0  
2020-01-06  241.490860   88800.0  
2020-01-07  242.849213   39600.0  
2020-01-08  247.337997  103500.0  

[5 rows x 135 columns]

In [5]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All Stocks

In [6]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for stock in stocks:
    # Use the stock data from our df
    df_stock = df[stock][['Close', 'Volume']]

    # Convert the stock to datetime if needed
    if not isinstance(df_stock.index, pd.DatetimeIndex):
        df_stock.index = pd.to_datetime(df_stock.index)

    # plot
    hist = go.Histogram(
        y=df_stock['Close'],
        x=df_stock['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_stock.index,
        y=df_stock['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {stock} Close Prices'
    )

    fig.show()